In [1]:
%%capture
!pip install faker
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install faker

In [2]:
import torch
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {DEVICE}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")

Device: cuda
CUDA Version: 12.1
Pytorch 2.1.2


In [3]:
import sys
import random
import string
import re
import time
import ast
import json

import pandas as pd
import numpy as np
from peft import PeftModel, PeftConfig
from transformers import (BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer,
                          pipeline, GenerationConfig)
from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from faker import Faker
from spacy.lang.en import English
from spacy.lang.ru import Russian

2024-03-21 17:50:14.724282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 17:50:14.724391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 17:50:14.884045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import torch, random
# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True

# This enables the inbuilt cudnn auto-tuner to find the best algorithm to use for current hardware
# If the model does not change and input sizes remains the same then you may benefit from setting torch.backends.cudnn.benchmark = True.
# However, if there are layers that are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a different number of times,
# then setting benchmark to True might stall the execution.
torch.backends.cudnn.benchmark = False

SEED = 12
# Seed the same seed to all 
def seed_everything(seed: int):
    Faker.seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(SEED)

In [5]:
import ctypes, gc, torch
libc = ctypes.CDLL("libc.so.6")

def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

In [6]:
def get_userid(length: int=16):
    """Generate userid - """
    userid = str(int(np.random.rand()*1_000_000_000))
    # Add the extra rand chars
    for _ in range(length):
        # Select random char or digital number (0-9)
        userid = userid + random.choice(string.ascii_letters + str(random.randint(0, 9))) 
    return userid

# Generate the personal url from social media 
def generate_fake_social_media_url(user_name: str):
    social_media_platforms = {
        'LinkedIn': 'linkedin.com/in/',
        'YouTube': 'youtube.com/c/',
        'Instagram': 'instagram.com/',
        'GitHub': 'github.com/',
        'Facebook': 'facebook.com/',
        'Twitter': 'twitter.com/',
        'Twitter_short': 't.co/',
        'VK': 'vk.com/',
        'Telegram': 't.me/'
    }
    platform, domain = random.choice(list(social_media_platforms.items()))
    fake_url = f'https://{domain}{user_name}'
    return fake_url

def generate_username(first_name: str, last_name: str, fake_user_name: str):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]    
    if random.choice([False, True]):
        username = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}{random.randint(1,999)}"
    else:
        username = fake_user_name
    return username

def generate_email(first_name: str, last_name: str, faker):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]
    if random.choice([False, True]):
        email = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}@{faker.domain_name()}"
    else:
        email = faker.email()
    return email

def generate_entity_info():
    """Generates all the user info (name, eamil addresses, phone number, etc) together """
    # Select the student country to generate the user info based on the country
    COUNTRIES = [
#                  "en_US", "en_US", "en_US", "en_US", "en_US", "en_US",
#                  "de_DE", "it_IT", "es_ES", "da_DK", "fr_FR", "vi_VN",
                 "ru_RU"    
                ]
    country = random.choice(COUNTRIES)
    faker = Faker(country)
    if random.choice([True, False]):
        first_name = faker.first_name_male()
        last_name = faker.last_name_male()
    else:
        first_name = faker.first_name_female()
        last_name = faker.last_name_female()
    fake_url = generate_fake_social_media_url(Faker('en_US').user_name())
    entity = {}
    entity['COUNTRY'] = country
    entity['ID_NUM'] = str(get_userid())
    entity['NAME'] = first_name + " "+  last_name 
    entity['EMAIL'] = generate_email(Faker('en_US').first_name(), Faker('en_US').last_name(), Faker('en_US'))
    entity['USERNAME'] = Faker('en_US').user_name()
    entity['PHONE_NUM'] = str(faker.phone_number().replace(" ", ""))
    entity['URL_PERSONAL'] = str(fake_url)
    entity['CARD_NUMBER'] = str(''.join(str(random.randint(0, 9)) for _ in range(16)))
    entity['STREET_ADDRESS'] = str(faker.address()).replace("\n"," ")
    del faker
    clear_memory()
    print(entity)
    return entity   

In [7]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Твоя главная задача - писать эссе."

In [8]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [9]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

In [10]:
from pathlib import Path
Path("/kaggle/working/temp").mkdir(parents=True, exist_ok=True)

TOTAL = 100
entities = []
for i in tqdm(range(TOTAL)):
    entities.append(generate_entity_info())
df = pd.DataFrame(entities)
df = df.reset_index(drop=True)
# Save to the csv file
df.to_csv("/kaggle/working/temp/df.csv", index=False, encoding='UTF-8')
display(df.tail(10))
# sys.exit(0)

  0%|          | 0/100 [00:00<?, ?it/s]

{'COUNTRY': 'ru_RU', 'ID_NUM': '154162842EPSoajxvd9eRRZb0', 'NAME': 'Олимпиада Федотова', 'EMAIL': 'mvaughn@example.org', 'USERNAME': 'stanleydustin', 'PHONE_NUM': '8(525)309-31-85', 'URL_PERSONAL': 'https://linkedin.com/in/xsmith', 'CARD_NUMBER': '1671628598208281', 'STREET_ADDRESS': 'г. Биробиджан, пер. Главный, д. 267 стр. 3/9, 982082'}
{'COUNTRY': 'ru_RU', 'ID_NUM': '740049696bmwSRhpxGZlycbRc', 'NAME': 'Прасковья Авдеева', 'EMAIL': 'thomas_cruz@davidson.com', 'USERNAME': 'michael56', 'PHONE_NUM': '+71435558982', 'URL_PERSONAL': 'https://t.co/ebarnett', 'CARD_NUMBER': '9183214064759087', 'STREET_ADDRESS': 'ст. Руза, пер. Заводской, д. 16, 391832'}
{'COUNTRY': 'ru_RU', 'ID_NUM': '263315015pDxmMrlYGphIjuOo', 'NAME': 'Бажен Игнатов', 'EMAIL': 'jessica.palmer@clark-campbell.info', 'USERNAME': 'kallen', 'PHONE_NUM': '8(852)339-04-52', 'URL_PERSONAL': 'https://vk.com/ocooper', 'CARD_NUMBER': '5472749045873484', 'STREET_ADDRESS': 'д. Аша, пер. Пограничный, д. 9, 054138'}
{'COUNTRY': 'ru_RU

,COUNTRY,ID_NUM,NAME,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,CARD_NUMBER,STREET_ADDRESS
90,ru_RU,957417423szEEUIeJVzPGQEKl,Николай Пахомов,silvamarc@example.net,bkim,83797022082,https://t.me/thomasdoris,9808253623103472,"клх Гдов, пер. Казачий, д. 450, 103144"
91,ru_RU,405958342NzyaRZNDhoNPOHIw,Максим Волков,warrenvanessa@example.com,wallskara,8(970)553-55-32,https://twitter.com/tamara14,7800464526035550,"д. Каменск-Шахтинский, пр. Пограничный, д. 4/4..."
92,ru_RU,946555822qRDkczzmpJlFZTHp,Валентина Шарова,stephanie34@example.net,jmoore,+76529030257,https://t.me/cortezsandra,1047800226162019,"ст. Кировск (Мурм.), пр. Портовый, д. 7/5 к. 5..."
93,ru_RU,857190558fLHDcgBvfJxwUDJr,Анна Гаврилова,jhernandez@example.net,kenneth67,8(378)584-19-80,https://linkedin.com/in/nrogers,4462826345717356,"ст. Южно-Сахалинск, бул. Коллективный, д. 3/2 ..."
94,ru_RU,688927666dCcWhcFBCBHO5nBo,Ермил Комаров,kimberly.bowman@ryan-morgan.org,randall65,+70759874153,https://t.me/mmartin,4083441141575186,"клх Троицк (Моск.), бул. Волкова, д. 9/6 стр. ..."
95,ru_RU,3287968btwYYVgBwwQNGfEw,Лидия Алексеева,hkeller@example.org,jeremy87,+7(280)965-65-31,https://linkedin.com/in/sarah55,7596113450803478,"п. Двинской, ш. Тамбовское, д. 7/9 к. 4, 104780"
96,ru_RU,900103499xqArZNUOgWNBA0FD,Агап Колобов,jonathanmartin@example.org,christophergarza,+78073061768,https://linkedin.com/in/jdavis,9736463228042848,"г. Усть-Калманка, алл. Грибоедова, д. 6/3 стр...."
97,ru_RU,919864870haeQvktLIpvQgjbw,Казимир Федотов,josephhobbs@west.biz,virginia87,86988047103,https://t.me/hannahhamilton,8723052653335237,"ст. Алагир, алл. Мая 1, д. 7/2, 960865"
98,ru_RU,5909355CtKTuDbSYaXWQioM,Глафира Ершова,rita41@example.org,sherry15,8(518)635-10-80,https://t.co/spencer40,4968548323981775,"клх Шелагонцы, ул. Черемушки, д. 51 к. 3/6, 56..."
99,ru_RU,642926148FzhbShQgAtlcMyRH,Евлампий Михайлов,marylynch@davis-christian.com,yolanda08,+73478539843,https://t.co/harry75,7902419811752230,"г. Обнинск, ш. Лесхозное, д. 565 стр. 86, 113592"


# Generate loop
The loop generate the texts based on each student's PII information, including name + email + street address + phone + username + user id + personal url.

All the generated texts are stored to `generate_df.csv`



In [11]:
def generate_on_user_info(row):

    first_name = row['NAME'].split()[0]
    last_name = row['NAME'].split()[1]
    email = row['EMAIL']
    phone_num = row['PHONE_NUM']
    address = row['STREET_ADDRESS']
    url = row['URL_PERSONAL']
    username = row['USERNAME']
    card_number = str(row['CARD_NUMBER'])
    
    prompt = f"""
    Ты писатель.
    Тебе дана личная информация студента, например,
    имя = {first_name} {last_name}
    электронная почта = {email}
    адрес проживания = {address}
    номер телефона = {phone_num}
    персональный сайт = {url}
    имя пользователя = {username}
    номер банковской карты = {card_number}
    Тебе необходимо написать эссе, которое будет включать всю перечисленную информацию.
    Не пропусти ни одного пункта.
"""
    
    return prompt

def generate_texts(model, tokenizer, df, num_essays):
    
    generated_df = df[:num_essays]
    # Generate the texts
    for i in tqdm(range(len(generated_df))):
        start = time.time()
        # Generate PII
        row = generated_df.loc[i]
        # Generate the texts using three prompts
        prompt = generate_on_user_info(row)
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    
        # Generate the outputs from prompt
        conversation = Conversation()
        conversation.add_user_message(prompt)
        prompt = conversation.get_prompt(tokenizer)
        generated_text = generate(model, tokenizer, prompt, generation_config)
        
        print(f"generated_text = {generated_text}" )
        generated_df.loc[i, 'generated_text'] = generated_text
        clear_memory()
        print(f"Complete the text for {i}-th student {time.time() - start: .1f} seconds")
    # Save generated_df to csv
    generated_df.to_csv("temp/generated_df.csv", index=False, encoding="UTF-8")

In [12]:
generate_texts(model, tokenizer, df, num_essays=100)

  0%|          | 0/100 [00:00<?, ?it/s]

generated_text = Олимпиада Федотова, студентка из города Биробиджан, имеет множество интересных особенностей. Ее электронная почта - mvaughn@example.org, а адрес проживания - г. Биробиджан, пер. Главный, дом 267, квартира 3/9, 982082. Она также обладает номером телефона 8(525)309-31-385, который можно использовать для связи с ней.

Если вы хотите узнать больше о Олимпиаде Федотовой, то есть возможность посмотреть ее персональный сайт, который расположен по адресу https://linkedin.com/in/xsmith. Там вы найдете все информацию о ее образовании, профессиональном опыте и других аспектах ее жизни.

Кроме того, Олимпиада Федотова использует номер банковской карты 167162859820882, что может быть полезно при обмене деньгами или покупках онлайн.

Имя пользователя Олимпиады Федотовой - stanleydustin, которое она использует в социальных сетях и других онлайн-платформах. Это помогает ей легко находиться и общаться с друзьями, коллегами и другими людьми, которые ей интересны.

В целом, Олимпиада Фед

/tmp/ipykernel_26/858035951.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_df.loc[i, 'generated_text'] = generated_text


Complete the text for 0-th student  154.8 seconds
generated_text = Прасковья Авдеева - это молодой студентка, которая активно использует современные технологии для обучения и общения с окружающими. Она имеет собственный персональный веб-сайт, где делится своими знаниями и опытом. Ее электронная почта - thomas_cruz@davidson.com, что говорит о том, что она может быть увлеченной различными темами и предметами.

Адрес проживания Прасковьи находится в городе Руза, на улице Заводская, дом 16. Это место, где она живет и работает над своим образованием. Её телефонный номер - +71435558982, и она всегда рада ответить на вопросы или помочь в решении каких-либо проблем.

Однако, стоит обратить внимание на то, что Прасковья обладает доступным номером банковской карты - 9183214064759. Это может быть полезно для ее семьи и друзей, которые могут использовать его для поддержки ее образования или других потребностей.

В целом, Прасковья Авдеева является талантливой студенткой, которая стремится к успеху

In [13]:
generated_df = pd.read_csv('/kaggle/working/temp/generated_df.csv')

In [14]:
generated_df.dtypes

COUNTRY           object
ID_NUM            object
NAME              object
EMAIL             object
USERNAME          object
PHONE_NUM         object
URL_PERSONAL      object
CARD_NUMBER        int64
STREET_ADDRESS    object
generated_text    object
dtype: object

In [15]:
generated_df['CARD_NUMBER'] = generated_df['CARD_NUMBER'].astype('str')

In [17]:
from spacy.lang.en import English
from spacy.lang.ru import Russian

en_tokenizer = English().tokenizer
ru_tokenizer = Russian().tokenizer
social_media_platforms = [
        'linkedin.com/in/',
        'youtube.com/c/',
        'instagram.com/',
        'github.com/',
        'facebook.com/',
        'twitter.com/',
        'vk.com/',
        't.me/',
        't.co/'
    ]

def check_url(lst, string):
    for item in lst:
        if item in string:
            return True
    return False

def tokenize_with_spacy(text, tokenizer=ru_tokenizer):
    tokenized_text = tokenizer(text)
    tokens = [token.text for token in tokenized_text]
    trailing_whitespace = [bool(token.whitespace_) for token in tokenized_text]
    return tokens, trailing_whitespace

# Update labels and boolean flags
def update_labels(i, token, label_type, labels, isFirst_flags):
    # Update the label
    if isFirst_flags[label_type]:
        labels[i] = label_type # Beginning of an entity
        isFirst_flags[label_type] = False
    else:
        labels[i] = label_type # Contiunity of an entity
    return labels, isFirst_flags

# Go through each token and assign name label ('NAME') if matched
def assign_name(row, tokens, labels, isFirst_flags):
    name = row['NAME'].split(' ')
    first_name = name[0].lower()
    last_name = name[1].lower()
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if first_name in token_ or last_name in token_:
            # Update the label
            labels, isFirst_flags = update_labels(i, token, 'NAME', labels, isFirst_flags)
    return labels, isFirst_flags

def assign_street_address(row, tokens, labels, isFirst_flags):
    # Remove punctuations from street address
    address = row['STREET_ADDRESS'].lower().translate(str.maketrans('', '', string.punctuation))
    # Tokenize the street address with spacy tokenizer
    address_tokens, _ = tokenize_with_spacy(address) 
    
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        label = labels[i]
        # Check if token is matched last name
        if label == 'O' and token_ in address_tokens:
            labels, isFirst_flags = update_labels(i, token_, 'STREET_ADDRESS',
                                                  labels, isFirst_flags)
    return labels, isFirst_flags
    
    
def assign_phone_number(row, tokens, labels, isFirst_flags):
    # Tokenize the street address with spacy tokenizer
    phone_tokens, _ = tokenize_with_spacy(row['PHONE_NUM'].lower(), ru_tokenizer)
    print(f"Search 'PHONE_NUM': {phone_tokens}")    
    
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        label = labels[i]
        # Check if token is matched last name
        if label == 'O' and token_ in phone_tokens:
            labels, isFirst_flags = update_labels(i, token_, 'PHONE_NUM',
                                                  labels, isFirst_flags)
    return labels, isFirst_flags

def assign_card_number(row, tokens, labels, isFirst_flags):
    # Tokenize the street address with spacy tokenizer
    card_tokens = row['CARD_NUMBER']
    print(f"Search 'CARD_NUMBER': {card_tokens}")    
    
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        label = labels[i]
        # Check if token is matched last name
        if label == 'O' and token_ in card_tokens:
            labels, isFirst_flags = update_labels(i, token_, 'CARD_NUMBER',
                                                  labels, isFirst_flags)
    return labels, isFirst_flags

def assign_email(row, tokens, labels, isFirst_flags):
    # Tokenize the street address with spacy tokenizer
    email_tokens, _ = tokenize_with_spacy(row['EMAIL'].lower(), en_tokenizer)
    print(f"Search 'EMAIL': {email_tokens}")    
    
    for i, token in enumerate(tokens):
        token_ = str(token).lower() 
        label = labels[i]
        # Check if token is matched last name
        if label == 'O' and token_ in email_tokens or '@' in token_:
            labels, isFirst_flags = update_labels(i, token_, 'EMAIL',
                                                  labels, isFirst_flags)
    return labels, isFirst_flags

def assign_url(row, tokens, labels, isFirst_flags):
    # Tokenize the street address with spacy tokenizer
    url_tokens, _ = tokenize_with_spacy(row['URL_PERSONAL'].lower(), en_tokenizer)
    print(f"Search 'URL_PERSONAL': {url_tokens}")    
    
    for i, token in enumerate(tokens):
        token_ = str(token).lower() 
        label = labels[i]
        # Check if token is matched last name
        if label == 'O' and token_ in url_tokens or check_url(social_media_platforms, token_):
            labels, isFirst_flags = update_labels(i, token_, 'URL_PERSONAL',
                                                  labels, isFirst_flags)
    return labels, isFirst_flags

# Assign labels for other types
def assign_other_label_types(row, label_type, tokens, labels, isFirst_flags):
    label_value = row[label_type].lower()   
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if label_value in token_:
            # Update the label
            labels, isFirst_flags = update_labels(i, token, label_type, labels, isFirst_flags)
    return labels, isFirst_flags


def assign_labels(row, tokens):
    label_types = ['NAME', 'EMAIL', 'USERNAME',
                   'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS', 'CARD_NUMBER']
    # Assign "O" to labels by default
    labels = ['O' for _ in tokens] 
    # Create a boolean flag list to track if a label type start the text.
    isFirst_flags = {label_type: True for label_type in label_types}
    # Go through each token and check if the label appear in the token
    # All token and label values are lower case for comparison
    for label_type in label_types:
        if label_type == 'NAME':
            labels, isFirst_flags = assign_name(row, tokens, labels, isFirst_flags)
        elif label_type == 'STREET_ADDRESS':
            labels, isFirst_flags = assign_street_address(row, tokens, labels, isFirst_flags) 
        elif label_type == 'PHONE_NUM':
            labels, isFirst_flags = assign_phone_number(row, tokens, labels, isFirst_flags)
        elif label_type == 'CARD_NUMBER':
            labels, isFirst_flags = assign_card_number(row, tokens, labels, isFirst_flags)
        elif label_type == 'EMAIL':
            labels, isFirst_flags = assign_email(row, tokens, labels, isFirst_flags)
        elif label_type == 'URL_PERSONAL':
            labels, isFirst_flags = assign_url(row, tokens, labels, isFirst_flags)
        else:
            labels, isFirst_flags = assign_other_label_types(row, label_type, tokens,
                                                             labels, isFirst_flags)
    
    return labels
   
# Map the label to token
def get_token_map(tokens, labels):
    token_map = []
    for i, label in enumerate(labels):
        if label != 'O':
            token_map.append({label: tokens[i]})
    return token_map

In [18]:
results = []
doc_id = 1_111_111_000 # document id 
for i in range(len(generated_df)):
    row = generated_df.loc[i]
    full_text = row["generated_text"]
    print(f"full_text = {full_text}")
    # Tokenize the text using spacy tokenizer
    tokens, trailing_whitespace = tokenize_with_spacy(full_text)
    labels = assign_labels(row, tokens)
    token_map = get_token_map(tokens, labels)
    doc_id += 1
    result = {
        'document': doc_id, 
        'full_text': full_text,
        'tokens': tokens, 
        'trailing_whitespace': trailing_whitespace,
        'labels': labels,
        'token_map': token_map
    }
    results.append(result)
    
# Save to temp fold for verification
tmp_df = pd.DataFrame(results, columns=['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels', 'token_map'])
tmp_df.to_csv("temp/pii_dataset.csv", index=False, encoding="UTF-8")

full_text = Олимпиада Федотова, студентка из города Биробиджан, имеет множество интересных особенностей. Ее электронная почта - mvaughn@example.org, а адрес проживания - г. Биробиджан, пер. Главный, дом 267, квартира 3/9, 982082. Она также обладает номером телефона 8(525)309-31-385, который можно использовать для связи с ней.

Если вы хотите узнать больше о Олимпиаде Федотовой, то есть возможность посмотреть ее персональный сайт, который расположен по адресу https://linkedin.com/in/xsmith. Там вы найдете все информацию о ее образовании, профессиональном опыте и других аспектах ее жизни.

Кроме того, Олимпиада Федотова использует номер банковской карты 167162859820882, что может быть полезно при обмене деньгами или покупках онлайн.

Имя пользователя Олимпиады Федотовой - stanleydustin, которое она использует в социальных сетях и других онлайн-платформах. Это помогает ей легко находиться и общаться с друзьями, коллегами и другими людьми, которые ей интересны.

В целом, Олимпиада Федотова

# Save to CSV

In [20]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pii_df = pd.DataFrame(results, columns=['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])
pii_df.to_csv("pii_dataset.csv", index=False, encoding="UTF-8")
pii_df.head(10)

,document,full_text,tokens,trailing_whitespace,labels
0,1111111001,"Олимпиада Федотова, студентка из города Биробиджан, имеет множество интересных особенностей. Ее электронная почта - mvaughn@example.org, а адрес проживания - г. Биробиджан, пер. Главный, дом 267, квартира 3/9, 982082. Она также обладает номером телефона 8(525)309-31-385, который можно использовать для связи с ней.\n\nЕсли вы хотите узнать больше о Олимпиаде Федотовой, то есть возможность посмотреть ее персональный сайт, который расположен по адресу https://linkedin.com/in/xsmith. Там вы найдете все информацию о ее образовании, профессиональном опыте и других аспектах ее жизни.\n\nКроме того, Олимпиада Федотова использует номер банковской карты 167162859820882, что может быть полезно при обмене деньгами или покупках онлайн.\n\nИмя пользователя Олимпиады Федотовой - stanleydustin, которое она использует в социальных сетях и других онлайн-платформах. Это помогает ей легко находиться и общаться с друзьями, коллегами и другими людьми, которые ей интересны.\n\nВ целом, Олимпиада Федотова - это молодая и талантливая студентка, которая активно участвует в жизни своего города и стремится развиваться как профессионально, так и лично.","[Олимпиада, Федотова, ,, студентка, из, города, Биробиджан, ,, имеет, множество, интересных, особенностей, ., Ее, электронная, почта, -, mvaughn@example.org, ,, а, адрес, проживания, -, г., Биробиджан, ,, пер., Главный, ,, дом, 267, ,, квартира, 3/9, ,, 982082, ., Она, также, обладает, номером, телефона, 8(525)309, -, 31, -, 385, ,, который, можно, использовать, для, связи, с, ней, ., \n\n, Если, вы, хотите, узнать, больше, о, Олимпиаде, Федотовой, ,, то, есть, возможность, посмотреть, ее, персональный, сайт, ,, который, расположен, по, адресу, https://linkedin.com/in/xsmith, ., Там, вы, найдете, все, информацию, о, ее, образовании, ,, профессиональном, опыте, и, других, аспектах, ее, жизни, ., \n\n, Кроме, того, ...]","[True, False, True, True, True, True, False, True, True, True, True, False, True, True, True, True, True, False, True, True, True, True, True, True, False, True, True, False, True, True, False, True, True, False, True, False, True, True, True, True, True, True, False, False, False, False, False, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, False, True, False, ...]","[NAME, NAME, O, O, O, O, STREET_ADDRESS, O, O, O, O, O, O, O, O, O, PHONE_NUM, EMAIL, O, O, O, O, PHONE_NUM, O, STREET_ADDRESS, O, O, STREET_ADDRESS, O, O, STREET_ADDRESS, O, O, O, O, STREET_ADDRESS, O, O, O, O, O, O, PHONE_NUM, PHONE_NUM, PHONE_NUM, PHONE_NUM, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, URL_PERSONAL, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]"
1,1111111002,"Прасковья Авдеева - это молодой студентка, которая активно использует современные технологии для обучения и общения с окружающими. Она имеет собственный персональный веб-сайт, где делится своими знаниями и опытом. Ее электронная почта - thomas_cruz@davidson.com, что говорит о том, что она может быть увлеченной различными темами и предметами.\n\nАдрес проживания Прасковьи находится в городе Руза, на улице Заводская, дом 16. Это место, где она живет и работает над своим образованием. Её телефонный номер - +71435558982, и она всегда рада ответить на вопросы или помочь в решении каких-либо проблем.\n\nОднако, стоит обратить внимание на то, что Прасковья обладает доступным номером банковской карты - 9183214064759. Это может быть полезно для ее семьи и друзей, которые могут использовать его для поддержки ее образования или других потребностей.\n\nВ целом, Прасковья Авдеева является талантливой студенткой, которая стремится к успеху и развитию. Она активно исполь

In [21]:
pd.read_csv('/kaggle/working/pii_dataset.csv')

,document,full_text,tokens,trailing_whitespace,labels
0,1111111001,"Олимпиада Федотова, студентка из города Биробиджан, имеет множество интересных особенностей. Ее электронная почта - mvaughn@example.org, а адрес проживания - г. Биробиджан, пер. Главный, дом 267, квартира 3/9, 982082. Она также обладает номером телефона 8(525)309-31-385, который можно использовать для связи с ней.\n\nЕсли вы хотите узнать больше о Олимпиаде Федотовой, то есть возможность посмотреть ее персональный сайт, который расположен по адресу https://linkedin.com/in/xsmith. Там вы найдете все информацию о ее образовании, профессиональном опыте и других аспектах ее жизни.\n\nКроме того, Олимпиада Федотова использует номер банковской карты 167162859820882, что может быть полезно при обмене деньгами или покупках онлайн.\n\nИмя пользователя Олимпиады Федотовой - stanleydustin, которое она использует в социальных сетях и других онлайн-платформах. Это помогает ей легко находиться и общаться с друзьями, коллегами и другими людьми, которые ей интересны.\n\nВ целом, Олимпиада Федотова - это молодая и талантливая студентка, которая активно участвует в жизни своего города и стремится развиваться как профессионально, так и лично.","['Олимпиада', 'Федотова', ',', 'студентка', 'из', 'города', 'Биробиджан', ',', 'имеет', 'множество', 'интересных', 'особенностей', '.', 'Ее', 'электронная', 'почта', '-', 'mvaughn@example.org', ',', 'а', 'адрес', 'проживания', '-', 'г.', 'Биробиджан', ',', 'пер.', 'Главный', ',', 'дом', '267', ',', 'квартира', '3/9', ',', '982082', '.', 'Она', 'также', 'обладает', 'номером', 'телефона', '8(525)309', '-', '31', '-', '385', ',', 'который', 'можно', 'использовать', 'для', 'связи', 'с', 'ней', '.', '\n\n', 'Если', 'вы', 'хотите', 'узнать', 'больше', 'о', 'Олимпиаде', 'Федотовой', ',', 'то', 'есть', 'возможность', 'посмотреть', 'ее', 'персональный', 'сайт', ',', 'который', 'расположен', 'по', 'адресу', 'https://linkedin.com/in/xsmith', '.', 'Там', 'вы', 'найдете', 'все', 'информацию', 'о', 'ее', 'образовании', ',', 'профессиональном', 'опыте', 'и', 'других', 'аспектах', 'ее', 'жизни', '.', '\n\n', 'Кроме', 'того', ',', 'Олимпиада', 'Федотова', 'использует', 'номер', 'банковской', 'карты', '167162859820882', ',', 'что', 'может', 'быть', 'полезно', 'при', 'обмене', 'деньгами', 'или', 'покупках', 'онлайн', '.', '\n\n', 'Имя', 'пользователя', 'Олимпиады', 'Федотовой', '-', 'stanleydustin', ',', 'которое', 'она', 'использует', 'в', 'социальных', 'сетях', 'и', 'других', 'онлайн', '-', 'платформах', '.', 'Это', 'помогает', 'ей', 'легко', 'находиться', 'и', 'общаться', 'с', 'друзьями', ',', 'коллегами', 'и', 'другими', 'людьми', ',', 'которые', 'ей', 'интересны', '.', '\n\n', 'В', 'целом', ',', 'Олимпиада', 'Федотова', '-', 'это', 'молодая', 'и', 'талантливая', 'студентка', ',', 'которая', 'активно', 'участвует', 'в', 'жизни', 'своего', 'города', 'и', 'стремится', 'развиваться', 'как', 'профессионально', ',', 'так', 'и', 'лично', '.']","[True, False, True, True, True, True, False, True, True, True, True, False, True, True, True, True, True, False, True, True, True, True, True, True, False, True, True, False, True, True, False, True, True, False, True, False, True, True, True, True, True, True, False, False, False, False, False, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, False, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, False, False, False, True, True, True, True, True, True, True, True, True, False, True, True, True, True, False, True, True, True, False, False, False, True, False, True, True, True, True